In [1]:
from pathlib import Path

import pandas as pd
from sklearn.metrics import cohen_kappa_score

SEED = 420

In [2]:
df = pd.read_csv("tsvs/rhyme_scheme_counts_clean.tsv", sep="\t")
df

,scheme,count
0,ABAB,1142
1,ABCB,749
2,AABB,611
3,AABCCB,360
4,ABBA,181
...,...,...
485,ABCDEFGHG,1
486,ABCDCEA,1
487,ABCDBEC,1
488,ABAAC,1


In [3]:
top_ten = df[:10].copy()
top_ten

,scheme,count
0,ABAB,1142
1,ABCB,749
2,AABB,611
3,AABCCB,360
4,ABBA,181
5,AABBCC,162
6,AAA,152
7,ABAAB,73
8,AABCBC,70
9,ABABCC,58


## Create subset for inter-annotator agreement

In [4]:
# Find how many stanzas from each rhyme scheme
top_ten["selection"] = top_ten["count"]/100*2.8
top_ten["selection"] = top_ten["selection"].apply(round)
print(sum(top_ten["selection"]))
top_ten

100


,scheme,count,selection
0,ABAB,1142,32
1,ABCB,749,21
2,AABB,611,17
3,AABCCB,360,10
4,ABBA,181,5
5,AABBCC,162,5
6,AAA,152,4
7,ABAAB,73,2
8,AABCBC,70,2
9,ABABCC,58,2


In [5]:
poem_df = pd.read_csv(f"tsvs/tita_rhymes_poems.tsv", sep="\t")

subset = poem_df.loc[poem_df["rhyme scheme"] == ""]

for e in top_ten.itertuples():
    _df = poem_df.loc[poem_df["rhyme scheme"] == e.scheme]
    selection = _df.sample(n=e.selection, random_state=SEED)
    subset = pd.concat((subset, selection))

In [6]:
subset

,rhyme scheme,stanza,filename,stanza number (in file)
2249,ABAB,"Å, så ramme også meg din vrede.\nLa meg dø! Ta...",2011042608056_9_annotated.txt,12
4752,ABAB,I jubel flyr toget avsted.\nSe grensen! En lin...,2011102708082_46_annotated.txt,1
2471,ABAB,"Sangen har evighet, derfor den skyter\nfortid ...",2006081000055_38_annotated.txt,1
3042,ABAB,Har jeg ikke sett dem de hellige trær\nher i s...,2014102306006_38_annotated.txt,16
2138,ABAB,"Jeg ser hans dag. Jeg ser ham komme,\nden seir...",2011041408047_30_annotated.txt,19
...,...,...,...,...
1082,ABAAB,Dog — nå er hans time inne.\nLyset er til jord...,2011041408091_22_annotated.txt,19
4064,AABCBC,Det har Paiulus ikke glemt.\nDerfor er hans ne...,2011041408091_17_annotated.txt,24
4950,AABCBC,Så til Rom! Det er hans håb forvisst\nalt tilt...,2011041408091_27_annotated.txt,29
4478,ABABCC,Uskyldig! Uskyldig! Jeg gjorde ille!\nTa sølve...,2011042608056_38_annotated.txt,42


## Shuffle subset so not all stanzas with the same scheme come after each other

In [7]:
subset = subset.sample(frac=1, random_state=SEED).reset_index(drop=True)
subset

,rhyme scheme,stanza,filename,stanza number (in file)
0,ABAB,"Regn, regn!\nropte skog og mark,\nregn, regn!\...",2006081000055_59_annotated.txt,5
1,ABCB,Vel klager det trette hjerte\ni mismot så mang...,2011041408047_20_annotated.txt,4
2,ABCB,Evig står nå striden\nom den usle grav\nmellom...,2014102306006_110_annotated.txt,5
3,ABAB,"Overalt han har spioner —\nkongen lytter, lure...",2011041408047_21_annotated.txt,1
4,ABCB,"Hva nevnes dog den smerte,\nden sorgens bitre ...",2011042608056_1_annotated.txt,4
...,...,...,...,...
95,ABAB,"I fjellets ly der stein har mæle,\nog dalen åp...",2012032024065_5_annotated.txt,3
96,AABB,Til slutning svevde over vang\nde klare kirkek...,2012032024065_11_annotated.txt,57
97,ABAB,Eller vanens sløve makt\nsom den beste vilje l...,2011041408091_22_annotated.txt,7
98,AABCCB,Eller båter med lina mens leken går vill\nsom ...,2007011001018_7_annotated.txt,6


## Write subset to file

In [8]:
subset_text = "\n\n".join(subset["stanza"])
with open("poem_subset/poem_subset", "w+") as f:
    f.write(subset_text)

## Read second annotator annotations

In [9]:
second_annot = pd.read_csv("tsvs/petter_rhymes_poems.tsv", sep="\t")
second_annot

,rhyme scheme,stanza,filename,stanza number (in file)
0,ABAB,"Regn, regn!\nropte skog og mark,\nregn, regn!\...",poem_su_annotated.txt,0
1,ABCB,Vel klager det trette hjerte\ni mismot så mang...,poem_su_annotated.txt,1
2,ABCB,Evig står nå striden\nom den usle grav\nmellom...,poem_su_annotated.txt,2
3,ABAB,"Overalt han har spioner —\nkongen lytter, lure...",poem_su_annotated.txt,3
4,ABCB,"Hva nevnes dog den smerte,\nden sorgens bitre ...",poem_su_annotated.txt,4
...,...,...,...,...
95,ABAB,"I fjellets ly der stein har mæle,\nog dalen åp...",poem_su_annotated.txt,95
96,AABB,Til slutning svevde over vang\nde klare kirkek...,poem_su_annotated.txt,96
97,ABAB,Eller vanens sløve makt\nsom den beste vilje l...,poem_su_annotated.txt,97
98,AABCCB,Eller båter med lina mens leken går vill\nsom ...,poem_su_annotated.txt,98


## Cohens kappa complete match

In [10]:
cohen_kappa_score(subset["rhyme scheme"], second_annot["rhyme scheme"])

0.9262899262899262

## Cohens kappa on letter level

In [11]:
a1 = [c for code in subset["rhyme scheme"] for c in code]
a2 = [c for code in second_annot["rhyme scheme"] for c in code]
cohen_kappa_score(a1, a2)

0.9586943415726467

## Simple match

In [12]:
agree = sum(c1 == c2 for c1, c2 in zip(subset["rhyme scheme"], second_annot["rhyme scheme"]))
disagree = sum(c1 != c2 for c1, c2 in zip(subset["rhyme scheme"], second_annot["rhyme scheme"]))

In [13]:
agree, disagree, agree+disagree

(94, 6, 100)

In [14]:
disagree1 = subset.loc[subset["rhyme scheme"] != second_annot["rhyme scheme"]]
disagree2 = second_annot.loc[second_annot["rhyme scheme"] != subset["rhyme scheme"]]
for e1, e2 in zip(disagree1.itertuples(), disagree2.itertuples()):
    print(e1._1)
    print(e2._1)
    print(e1.stanza)
    print("\n")

AABCBC
AABCCB
Så til Rom! Det er hans håb forvisst
alt tiltross, å nå sitt mål til sist.
Alt er ferdig. Ved den nære strand
seilklart ligger skipet som skal bære
Paulus over havets bølgehære
mot det første av hans lengslers land.


AABB
ABCC
Så vet Elias. Milde ord
for folket nå er gift og mord. 
Den kjærlighet som her har verd,
ei kjæle må, men bruke sverd.


ABAB
ABBA
I den falt gnisten av min moders lyse
og varme sjel; og ekteskapets lykke
ble ei alene deres alders-smykke;
men den vil lenge etter døden lyse.


ABCB
ABAC
Han vet ikke ennu på hvilken vis
de frelses, kun ett han vet:
at Israels folk skal synge Guds pris
før neste dags sol går ned.


ABBA
ABBC
Og deres gjerning krever billig bod.
Så slippes koblet løs, og byttet jages,
fra hjem og arne menn og kvinner drages.
Og Rom blir drukken av de kristnes blod.


AABCCB
AABCCD
Ja løsner så berget sitt vingede skred
og storflokken styrter fra fjellveggen ned
med småbåter like innunder
Med englenes hærskarer liknes da må
de fjærede s